In [44]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#Enter details of your AzureML workspace
subscription_id = 'fe38c376-b42a-4741-9e7c-f5d7c31e5873'
resource_group = 'yelizkilinc-rg'
workspace = 'aml-prod'

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [3]:
%pip install -U 'azureml-rag[faiss]>=0.1.11'

  Using cached azureml_rag-0.1.14-py3-none-any.whl (190 kB)
  Using cached azureml_dataprep-4.11.6-py3-none-any.whl (38.2 MB)
  Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 61.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 43.2 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of azureml-fsspec to determine which version is compatible with other requirements. This could take a while.
  Using cached azureml_fsspec-1.1.1-py3-none-any.whl (10 kB)
  Using cached azure_storage_blob-12.13.0-py3-none-any.whl (377 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
%%writefile workspace.json
{
"subscription_id" : "fe38c376-b42a-4741-9e7c-f5d7c31e5873",
"resource_group" : "yelizkilinc-rg",
"workspace_name" : "aml-prod"
}

Writing workspace.json


In [7]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    # ml_client = MLClient(
    #     credential=credential,
    #     subscription_id="",
    #     resource_group_name="",
    #     workspace_name=""
    # )

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f5d2dbcbb80>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


In [10]:
from azureml.rag.utils.connections import (
    get_connection_by_name_v2,
    create_connection_v2,
)

aoai_connection_name = "promptflowtest"
aoai_connection = get_connection_by_name_v2(ws, aoai_connection_name)
aoai_connection_id = aoai_connection["id"]

In [11]:
aoai_connection_id

'/subscriptions/fe38c376-b42a-4741-9e7c-f5d7c31e5873/resourceGroups/yelizkilinc-rg/providers/Microsoft.MachineLearningServices/workspaces/aml-prod/connections/promptflowtest'

In [19]:
import uuid

AOAI_endpoint = "https://openaiykus.openai.azure.com/" #os.environ.get("LOCAL_SECRET", default = str(uuid.uuid4())) # Use random UUID as a substitute for real secret.
key_value="9ed9191bbdae445cbe69af2709668995"
keyvault = ws.get_default_keyvault()
keyvault.set_secret(name="aoai-endpoint", value = AOAI_endpoint)


In [20]:
keyvault.set_secret(name="key",value=key_value)

In [21]:
aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")

In [14]:
import openai

In [22]:
# Set up Azure OpenAI
openai.api_type = "azure"
openai.api_base = aoai_endpoint # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2022-12-01"
openai.api_key = aoai_key

In [39]:
prompt_prefix = """ 
  Extract keywords from only this text: 
"""

prompt = prompt_prefix + "Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn."
prompt

' \n  Extract keywords from only this text: \nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.'

In [42]:
response = openai.Completion.create(
  deployment_id="gpt-35-turbo", # has to be deployment_id
  prompt= prompt,
  temperature=1,
  max_tokens=10,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0
)


In [43]:
# print response
response['choices'][0]['text']

'  Its profits were buoyed by one-off gains'